In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('./data/train/cats/cat.1.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='./data/preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely
        

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

from tensorflow.keras import backend as K

img_width, img_height=150,150
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)


In [6]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        1

In [8]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        './data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        './data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [9]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/50
125/125 [==============================] - 68s 545ms/step - loss: 0.7232 - accuracy: 0.5345 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 2/50
125/125 [==============================] - 66s 531ms/step - loss: 0.6740 - accuracy: 0.5945 - val_loss: 0.6135 - val_accuracy: 0.6737
Epoch 3/50
125/125 [==============================] - 65s 516ms/step - loss: 0.6264 - accuracy: 0.6510 - val_loss: 0.6412 - val_accuracy: 0.5987
Epoch 4/50
125/125 [==============================] - 65s 517ms/step - loss: 0.6025 - accuracy: 0.6705 - val_loss: 0.6054 - val_accuracy: 0.6762
Epoch 5/50
125/125 [==============================] - 65s 521ms/step - loss: 0.5906 - accuracy: 0.6990 - val_loss: 0.5485 - val_accuracy: 0.7287
Epoch 6/50
125/125 [==============================] - 65s 520ms/step - loss: 0.5638 - accuracy: 0.7115 - val_loss: 0.5684 - val_accuracy: 0.7025
Epoch 7/50
125/125 [==============================] - 65s 517ms/step - loss: 0.5456 - accuracy: 0.7190 - val_loss: 0.5543 - val_ac

# using the bottleneck feature of the VGG16

In [13]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications


In [17]:
img_width, img_height = 150, 150

top_model_weights_path = './bottleneck_fc_model.h5'
train_data_dir = './data/train'
validation_data_dir = './data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [21]:

def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('./bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('./bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


In [30]:
def train_top_model():
    train_data = np.load(open('./bottleneck_features_train.npy','rb'))
    train_labels = np.array(
        [0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))

    validation_data = np.load(open('./bottleneck_features_validation.npy','rb'))
    validation_labels = np.array(
        [0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [31]:
save_bottlebeck_features()
train_top_model()

Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 3s 1ms/sample - loss: 0.7224 - accuracy: 0.7540 - val_loss: 0.4710 - val_accuracy: 0.7900
Epoch 2/50
2000/2000 [==============================] - 2s 876us/sample - loss: 0.3605 - accuracy: 0.8515 - val_loss: 0.4983 - val_accuracy: 0.8125
Epoch 3/50
2000/2000 [==============================] - 2s 873us/sample - loss: 0.3007 - accuracy: 0.8805 - val_loss: 0.3076 - val_accuracy: 0.8888
Epoch 4/50
2000/2000 [==============================] - 2s 874us/sample - loss: 0.2740 - accuracy: 0.8950 - val_loss: 0.2867 - val_accuracy: 0.8963
Epoch 5/50
2000/2000 [==============================] - 2s 914us/sample - loss: 0.2512 - accuracy: 0.9060 - val_loss: 0.2659 - val_accuracy: 0.9013
Epoch 6/50
2000/2000 [==============================] - 2s 950us/sample - loss: 0.2153 - accuracy: 0.9170 - val_loss: 0.2978 - val_accuracy: 0.9000
Epoch 7/50
2000/2000 [==============================] - 2s 964us/sa

# Fine-tuning the top layers of a a pre-trained network